In [ ]:
# Mecabのインストール
!apt install aptitude
!aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y
!pip install mecab-python3==0.7

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  aptitude-common libcwidget4 libsigc++-2.0-0v5 libxapian30
Suggested packages:
  apt-xapian-index aptitude-doc-en | aptitude-doc debtags tasksel libcwidget-dev xapian-tools
The following NEW packages will be installed:
  aptitude aptitude-common libcwidget4 libsigc++-2.0-0v5 libxapian30
0 upgraded, 5 newly installed, 0 to remove and 31 not upgraded.
Need to get 3,838 kB of archives.
After this operation, 17.3 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 aptitude-common all 0.8.13-3ubuntu1 [1,719 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 libsigc++-2.0-0v5 amd64 2.10.4-2ubuntu3 [12.1 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libcwidget4 amd64 0.5.18-5build1 [306 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libxapian30 amd64 1

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import MeCab
import itertools
from operator import itemgetter
def syori(file):
  data_array = []

  with open(f"{file}_data.csv", 'r', encoding='utf-8') as file:
      reader = csv.reader(file)
      for row in reader:
        data_array.append(row[1])

  sentences=data_array
  #print(sentences)
  # MeCabのTaggerを生成
  tagger = MeCab.Tagger()

  # 形態素解析結果から助詞を取り除く関数
  def remove_particles(parsed_text):
      lines = parsed_text.split('\n')
      filtered_words = [line.split('\t')[0] for line in lines if '\t' in line and line.split('\t')[1].startswith('助詞') is False]
      return ' '.join(filtered_words)

  # 形態素解析と助詞の削除を行い、結果をwakatiリストに格納
  wakati = []
  for sentence in sentences:
      parsed_text = tagger.parse(sentence).strip()
      filtered_text = remove_particles(parsed_text)
      wakati.append(filtered_text)

  # TF-IDFの計算
  vectorizer = TfidfVectorizer(token_pattern=u'(?u)\\b\\w+\\b')
  tfidf = vectorizer.fit_transform(wakati)


  # コサイン類似度の計算
  similarity_matrix = cosine_similarity(tfidf)
  #print("Features:", vectorizer.get_feature_names_out())
  # 類似度の表示
  numbers = list(range(1, len(sentences)))

  # 全通りの組み合わせを生成
  combinations = list(itertools.combinations(numbers, 2))
  # TF-IDFの特徴名を取得
  feature_names = vectorizer.get_feature_names_out()

  # TF-IDFの重要度を取得
  tfidf_array = tfidf.toarray()

  # 各文章ごとの単語の重要度をまとめる
  word_importance_per_sentence = [dict(zip(feature_names, values)) for values in tfidf_array]

  # 各単語の総重要度を計算
  total_word_importance = {}
  for importance_dict in word_importance_per_sentence:
      for word, importance in importance_dict.items():
          total_word_importance[word] = total_word_importance.get(word, 0) + importance

  # 重要度の高い順にソート
  sorted_word_importance = sorted(total_word_importance.items(), key=itemgetter(1), reverse=True)

  # 上位N個の単語をピックアップ
  top_n_words = 100  # 上位N個を指定
  picked_up_words = [(word, importance) for word, importance in sorted_word_importance if len(word) > 1][:top_n_words]
  # CSVファイルに結果を保存
  output_file_path = f'{file}_picked_up_words.csv'
  with open(output_file_path, 'w', newline='', encoding='utf-8') as csvfile:
      fieldnames = ['番号', '単語', '重要度']
      writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
      writer.writeheader()
      delete_list=["ます","です","ない","こと","する","ある","いる","その","なっ","だっ","この","そんな","いく","あり","なる","ため"]
      # ピックアップした単語をCSVに書き込む
      for i, (word, importance) in enumerate(picked_up_words, start=1):
          if word in delete_list:
            continue
          else:
            writer.writerow({'番号': i, '単語': word, '重要度': importance})

  #print(f"結果が {output_file_path} に保存されました。")

  # 全通りの組み合わせを表示
  ruizido_ave=[]
  for combo in combinations:
    ruizido_ave.append(similarity_matrix[combo[0]][combo[1]])
#  print("sum=",sum(ruizido_ave)/len(ruizido_ave))
  return sum(ruizido_ave)/len(ruizido_ave)


In [ ]:
#あらすじのファイル作成
import requests
import json
import csv
def output(file,number):
  if number==0:
    gente="gente"
  else:
    gente="notgenre"
  url = "http://api.syosetu.com/novelapi/api/"

  # 小説名、作者名、Nコード、キーワードを出力
  # キーワードに「著作権フリー」を含むものを出力
  # json形式で出力
  payload = {'of': 's', 'out':'json',gente:"101-201-202","lim":"500"}

  response = requests.get(url, payload).json()

  title_list=[]
  for i  in range(len(response)):
      if i==0:
          continue
      title_list.append(response[i]["story"])
  with open(f'{file}_data.csv', 'w', newline='', encoding='utf-8') as file:
      writer = csv.writer(file)
      writer.writerow(['Index', 'Story'])
      for index, story in enumerate(title_list, start=1):
          writer.writerow([index, story])


In [ ]:
f_ave=[]
n_ave=[]
for i in range(10):
  output("fantasy", 0)
  f_ave.append(syori("fantasy"))

  output("not", 1)
  n_ave.append( syori("not"))
print("f",f_ave)
print("n",n_ave)
print("異世界系の類似度",sum(f_ave)/len(f_ave))
print("その他のの類似度",sum(n_ave)/len(n_ave))

f [0.041338101901829544, 0.041338101901829544, 0.04117264602589043, 0.04117264602589043, 0.04117264602589043, 0.04117264602589043, 0.04117264602589043, 0.041172646025890436, 0.041172646025890436, 0.041172646025890436]
n [0.030498830587027244, 0.030434783497430742, 0.030434783497430746, 0.030434783497430746, 0.030434783497430746, 0.030434783497430746, 0.030434783497430742, 0.030434783497430742, 0.030434783497430742, 0.030434783497430742]
異世界系の類似度 0.04120573720107826
その他のの類似度 0.030441188206390392
